In [2]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.visualize as visualize
import seaborn as sns
import matplotlib.pyplot as plt

import petab
import numpy as np
from scipy.special import gammaln
import pickle
import os

# path of the directory
d = os.getcwd()
# import to petab
petab_problem = petab.Problem.from_yaml(
    "Borghans_BiophysChem1997/Borghans_BiophysChem1997.yaml")
%matplotlib inline

In [ ]:
def analytical_b(t, a0, b0, k1, k2):
    return (k2 - k2 * np.exp(-(k2 + k1) * t)) / (k2 + k1)

def simulate_model(x, tvec):
    # assign parameters
    k1, k2 = x
    # define initial conditions
    a0 = 1
    b0 = 0.01
    # simulate model
    simulation = [analytical_b(t, a0, b0, k1, k2)
                   for t in tvec]
    return simulation

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

In [ ]:
def negative_log_marginal_posterior(x):
    """
    negative logarithmic marginalized posterior
    :param x: x_0 = k1, x_1 = k_2
    """
    
    x[:13] = np.power(10, x[:14])
    x[-3:] = np.power(10, x[-3:])
    
    # experimental data
    data = np.asarray(petab_problem.measurement_df.measurement)
    tvec = np.asarray(petab_problem.measurement_df.time)
    
    N = len(tvec)    
    
    _simulation = simulate_model(x, tvec)
    simulation = np.asarray(_simulation)
    
    C_1 = kappa * mu**2 + tau * z**2 + 2 * beta + np.sum(data**2) - (kappa * mu + np.sum(data))**2 /(N + kappa)
    
    C_2 = ((kappa * mu + np.sum(data)) * (np.sum(simulation)) - (N + kappa) * (tau * z + np.sum(simulation * data)))**2
    
    C_3 = (N + kappa) * ((N + kappa) * (tau + np.sum(data**2)) - np.sum(simulation)**2)
    
    C = 0.5 * C_1 - C_2 / (2 * C_3)
    
    mL_1 = (beta / C)**alpha / (scipy.special.)

    
    
    return -marg_likelihood

Now that we have the objective function defined, we need to create a pyPESTO problem.

In [ ]:
def marginal_sampling():
    """Creates a pyPESTO problem."""
    objective = pypesto.Objective(fun=negative_log_marginal_posterior)
    problem = pypesto.Problem(objective=objective,  # objective function
                              lb=[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0, 0, 0, 0.001, 0.001, 0.001],  # lower bounds
                              ub=[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 1, 1, 1, 100000, 100000, 100000],  # upper bounds
                              x_names=['K2', 'K_{par}', 'Ka', 'Kd', 'Kf', 'Kp', 'Ky', 'Kz', 'Vd', 'Vm2', 'Vm3', 'Vp', 'beta_{par}', 'epsilon_{par}', 'init_{A, state}', 'init_{Y, state}', 'init_{Z, state}', 'n_{par}', 'v0', 'v1'],  # parameter names
                              x_scales=['log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'lin', 'lin', 'lin', 'log10', 'log10', 'log10'])  # parameter scale
    return problem

In [ ]:
### Prior dependent paramters
# center the scaling parameter
mu=0
# std for scaling parameter --> higher = more constrained / lower = more relaxed
alpha=100
# center the sigma parameter
beta=0.1
# std for scaling parameter --> higher = more constrained / lower = more relaxed
kappa=0.01

# create the estimation problem
problem = marginal_sampling()

# MCMC chain length
n_samples= 100000

# call the sampler of choice
sampler = sample.AdaptiveMetropolisSampler()

Now we can perform the actual sampling.

In [ ]:
# Define number of runs
runs = 1

save_results = True # for testing just set to False

# Loop over n runs
for n in range(runs):
    # set initial random seed
    np.random.seed(n)
    # perform MCMC sampling
    result = sample.sample(problem, n_samples=n_samples, sampler=sampler,
                           x0=np.array([-1.2741, -0.6160]))
    # calculate effective sample size
    sample.effective_sample_size(result=result)

    # save the results as a pickle object
    if save_results:
        results = [result.sample_result, 'CR_MP']
        with open('Results/result_CR_MP_'+str(n)+'.pickle', 
                  'wb') as result_file:
            pickle.dump(results, result_file)

There are some built-in visualization functions that one can use.

In [ ]:
# Visualize the parameter trace
ax = visualize.sampling.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))

Based on our marginal sampling results we now want to sample $c$ and $\lambda$ based on the data:

\begin{align}
    \lambda \propto \operatorname{Gamma}(\alpha' = \alpha + N/2, \beta' = C). \label{eq precision from data}
\end{align}
for
\begin{align*}
    C \equiv  \frac{1}{2}\Biggl( \Biggl( \sum_{k = 1}^N (\overline{y_k} - h_k)^2 \Biggr) + \kappa \mu^2 + 2\beta \Biggr) -  \frac{1}{2(N + \kappa)} \Biggl(\Biggl(\sum_{k = 1}^N \overline{y_k} - h_k\Biggr)+ \kappa \mu \Biggr)^2
\end{align*}
and
\begin{align*}
    c \propto \mathcal{N} \left(\mu' = \frac{\left(\sum_{k = 1}^N \overline{y_k}-h_k \right) + \kappa \mu }{N + \kappa}, \hat{\lambda} = \lambda (N + \kappa) \right)
\end{align*}

In [ ]:
tvec = np.asarray(petab_problem.measurement_df.time)
N = len(tvec)
data_model = np.asarray(petab_problem.measurement_df.measurement)
fig, axs = plt.subplots(nrows=2, figsize=(12, 5))

def Constant(x):
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)

    res = data_model - simulation
    
    summand_1 = (np.sum(res**2) + kappa * mu **2 + 2 * beta)/2
    summand_2 = (1 / (2 * (N + kappa))) * (np.sum(res) + kappa * mu)**2
    
    return summand_1 - summand_2
    

Generator = np.random.default_rng()
results = pypesto.Result(marginal_sampling())


with open(d + '/Results/result_CR_MP_0.pickle', 'rb') as infile:
    results.sample_result = pickle.load(infile)[0]
                 
precision_list = np.zeros(np.shape(results.sample_result.trace_x)[1]) 

for index, data in enumerate(results.sample_result.trace_x[0, :, :]):
    shape = alpha + N/2
    scale = 1/Constant(data)
    precision_list[index] = Generator.gamma(shape, scale)

In [ ]:
axs[0] = sns.distplot(precision_list, rug=True, axlabel='precision')

In [ ]:
def mu_(x):
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)

    res = data_model - simulation
    result_ = np.sum(res) + kappa * mu
    return result_/(N + kappa)
    
offset_list = np.zeros(np.shape(results.sample_result.trace_x)[1]) 

for index, data in enumerate(results.sample_result.trace_x[0, :, :]):
    new_mu = mu_(data)
    new_sigmasquare = 1/((N + kappa)*precision_list[index])
    offset_list[index] = Generator.normal(new_mu, new_sigmasquare)

In [ ]:
axs[1] = sns.distplot(offset_list, rug=True, axlabel='offset')

In [ ]:
plt.tight_layout()
plt.savefig(fname=d + '\\offset_and_precision.png')